# Reducing Bias in Machine Learning Models - Pre-processing


#### Kristen Lo - BrainStation
---

### Table of Contents
- [Introduction](#intro)
- [Part 2: Pre-processing](#clean)
    - 1.1: [Methodology](#method)
    - 1.2: [Housekeeping](#house)
    - 1.3: [Pre-processing](#pre)
    - 1.4: [Handling Duplicates](#duplicates)
    - 1.5: [Data Type Conversion](#type)
    - 1.6: [Handling Missing Data](#missing) 
    - 1.7: [Categorizing Columns](#cat) 
- [Conclusion](#conc)


---
### <a id = 'intro'></a> Introduction

In this notebook, we will be doing some preprocessing to prepare our data for the ML pipleline. I will further discuss methodology later in the notebook. 

There is a need to be able to predict the hospital admission rates for diabetic patients. However, using traditional machine learning models can lead to health disparities caused by biased data which can be related to demographic data (ex. race, age, income, insurance etc). These biases need to be removed prior to modelling so that bias isn't introduced into the model. Building on the work of Raza, S. who aimed to predict, diagnose, and mitigate health disparities in hospital re-admission, my aim is to replicate the study performed by Raza and create my own model that's able to screen for biases and predict admission rates for diabetics visiting the ER. 


Data was sourced from all adult Emergency Department visits from March 2014 - July 2017 from one academic and two community emergency rooms, apart from the Yale New Haven Health system. These visits resulted in either admission to their respective hospital or discharge. 

There are a total of 972 variables that we extracted per patient visit from 560,486 patient visits. 

Courtesy of:
 "Hong WS, Haimovich AD, Taylor RA (2018) Predicting hospital admission at emergency department triage using machine learning. PLoS ONE 13(7): e0201016." (https://doi.org/10.1371/journal.pone.0201016)




-----

## <a id = 'clean'></a> Part 0: Pre-processing

Hyperglycemia is the medical term for having high blood glucose (sugar) levels. This an be due to a myriad of reasons, the most often being diabetes mellitus. The hemoglobin A1C (HbA1c) blood test is useful for diagnosing and monitoring diabetic patients. However, with our data set we will be looking at the median blood glucose 

![Pipeline](https://ars.els-cdn.com/content/image/1-s2.0-S2772442522000430-gr1.jpg)

From Raza,S. (2022)

---
#### <a id = 'method'></a> 1.2 Methodology

---
#### <a id = 'housekeeping'></a> 1.1 HouseKeeping 

Loading the necessary libraries

In [94]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import itertools
from scipy.stats import chi2_contingency

Loading the clean csv file

In [95]:
health_data = pd.read_csv('Data/clean_health_data_pt1.csv')

In [96]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_rows', None) #Show all rows

In [97]:
health_data.head()

,demo_age,demo_gender,demo_race,demo_employstatus,demo_insurance_status,disposition,huse_previousdispo,pmh_2ndarymalig,pmh_abdomhernia,pmh_acqfootdef,pmh_acrenlfail,pmh_acutecvd,pmh_acutemi,pmh_acutphanm,pmh_adltrespfl,pmh_alcoholrelateddisorders,pmh_amniosdx,pmh_anemia,pmh_aneurysm,pmh_anxietydisorders,pmh_artembolism,pmh_asppneumon,pmh_asthma,pmh_attentiondeficitconductdisruptivebeha,pmh_biliarydx,pmh_bladdercncr,pmh_blindness,pmh_bnignutneo,pmh_bonectcncr,pmh_bph,pmh_brainnscan,pmh_breastcancr,pmh_breastdx,pmh_brnchlngca,pmh_cardiaarrst,pmh_carditis,pmh_cataract,pmh_chfnonhp,pmh_chrkidneydisease,pmh_coaghemrdx,pmh_coloncancer,pmh_comabrndmg,pmh_complicdevi,pmh_complicproc,pmh_conduction,pmh_contraceptiv,pmh_copd,pmh_coronathero,pmh_crushinjury,pmh_cysticfibro,pmh_deliriumdementiaamnesticothercognitiv,pmh_developmentaldisorders,pmh_diabmelnoc,pmh_diabmelwcm,pmh_disordersusuallydiagnosedininfancych,pmh_diverticulos,pmh_dizziness,pmh_dminpreg,pmh_dysrhythmia,pmh_ecodesadverseeffectsofmedicalcare,pmh_ecodesfall,pmh_ecodesfirearm,pmh_ecodesmotorvehicletrafficmvt,pmh_ecodesotherspecifiedandclassifiable,pmh_encephalitis,pmh_endometrios,pmh_epilepsycnv,pmh_esophcancer,pmh_esophgealdx,pmh_eyeinfectn,pmh_femgenitca,pmh_feminfertil,pmh_fluidelcdx,pmh_fuo,pmh_fxarm,pmh_fxhip,pmh_fxskullfac,pmh_gangrene,pmh_gasduoulcer,pmh_gastritis,pmh_gastroent,pmh_giconganom,pmh_gihemorrhag,pmh_giperitcan,pmh_glaucoma,pmh_goutotcrys,pmh_guconganom,pmh_hdnckcancr,pmh_headachemig,pmh_hemorrpreg,pmh_hodgkinsds,pmh_hrtvalvedx,pmh_htn,pmh_htncomplicn,pmh_htninpreg,pmh_hyperlipidem,pmh_immunitydx,pmh_inducabortn,pmh_infectarth,pmh_influenza,pmh_infmalegen,pmh_intestinfct,pmh_intobstruct,pmh_intracrninj,pmh_jointinjury,pmh_kidnyrnlca,pmh_lateeffcvd,pmh_leukemias,pmh_liveribdca,pmh_lowbirthwt,pmh_lungexternl,pmh_lymphenlarg,pmh_maintchemr,pmh_maligneopls,pmh_meningitis,pmh_menstrualdx,pmh_miscellaneousmentalhealthdisorders,pmh_mooddisorders,pmh_mouthdx,pmh_ms,pmh_multmyeloma,pmh_mycoses,pmh_neoplsmunsp,pmh_nephritis,pmh_nonepithca,pmh_nonhodglym,pmh_nutritdefic,pmh_opnwndextr,pmh_osteoarthros,pmh_osteoporosis,pmh_otaftercare,pmh_otbnignneo,pmh_otbonedx,pmh_otcirculdx,pmh_otcomplbir,pmh_otconntiss,pmh_otdxbladdr,pmh_otdxkidney,pmh_otdxstomch,pmh_otendodsor,pmh_otfemalgen,pmh_othbactinf,pmh_othcnsinfx,pmh_othematldx,pmh_othercvd,pmh_othereardx,pmh_otheredcns,pmh_othereyedx,pmh_othergidx,pmh_othergudx,pmh_otherinjury,pmh_otherpregnancyanddeliveryincludingnormal,pmh_otherscreen,pmh_othfracture,pmh_othheartdx,pmh_othliverdx,pmh_othlowresp,pmh_othmalegen,pmh_othnervdx,pmh_othveindx,pmh_otinflskin,pmh_otitismedia,pmh_otjointdx,pmh_otnutritdx,pmh_otprimryca,pmh_otrespirca,pmh_otupprresp,pmh_otuprspin,pmh_ovariancyst,pmh_pancreascan,pmh_pancreasdx,pmh_paralysis,pmh_parkinsons,pmh_pathologfx,pmh_peripathero,pmh_peritonitis,pmh_personalitydisorders,pmh_phlebitis,pmh_pid,pmh_pleurisy,pmh_pneumonia,pmh_poisnotmed,pmh_precereoccl,pmh_prevcsectn,pmh_prolapse,pmh_prostatecan,pmh_pulmhartdx,pmh_rctmanusca,pmh_rehab,pmh_respdistres,pmh_retinaldx,pmh_rheumarth,pmh_schizophreniaandotherpsychoticdisorde,pmh_screeningandhistoryofmentalhealthan,pmh_septicemia,pmh_sexualinfxs,pmh_shock,pmh_sicklecell,pmh_skininfectn,pmh_skinmelanom,pmh_socialadmin,pmh_spincorinj,pmh_stomchcancr,pmh_substancerelateddisorders,pmh_suicideandintentionalselfinflictedin,pmh_superficinj,pmh_syncope,pmh_teethdx,pmh_testiscancr,pmh_thyroiddsor,pmh_tia,pmh_tonsillitis,pmh_ulceratcol,pmh_ulcerskin,pmh_unclassified,pmh_urinyorgca,pmh_uteruscancr,pmh_uti,pmh_varicosevn,pmh_viralinfect,pmh_whtblooddx,huse_n_edvisits,huse_n_admissions,triage_vital_hr,triage_vital_sbp,triage_vital_dbp,triage_vital_rr,triage_vital_temp,huse_n_surgeries,cc_abdominalcramping,cc_abdominaldistention,cc_abdominalpain,cc_abdominalpainpregnant,cc_abnormallab,cc_abscess,cc_addictionproblem,cc_alcoholintoxication,cc_alcoholproblem,cc_allergicreaction,cc_alteredmentalstatus,cc_animalbite,cc_ankleinjury,cc_anklepain,cc_anxiety,cc_arminjury,cc

---
#### <a id = 'pre'></a> 1.3 Pre-Processing

Before we can start dummying our data, we first need to clean up the categories. We will do this supercategory by supercategory. Let's first start with the `demographic` supercategory. 

In [98]:
demographic = health_data.filter(like='demo_').columns
print(f'The columns that make up the demographic super category are : {demographic}')

The columns that make up the demographic super category are : Index(['demo_age', 'demo_gender', 'demo_race', 'demo_employstatus',
       'demo_insurance_status'],
      dtype='object')


Let's now get the count of unique entries of each of these. 

In [99]:
# Initialize an empty dictionary to store counts for each column
counts_dict = {}

# Iterate through each column in the 'demographic' list
for column in demographic:
    # Get counts of unique entries for the current column
    counts = health_data[column].value_counts()
    
    # Store the counts in the dictionary
    counts_dict[column] = counts

# Print or use the counts for each column
for column, counts in counts_dict.items():
    print(f"Counts for {column}:")
    print(counts)
    print()

Counts for demo_age:
demo_age
50-59      23894
60-69      22236
70-79      17939
40-49      13088
80-89      13019
30-39       7397
90-99       4723
18-29       3195
100-109      136
Name: count, dtype: int64

Counts for demo_gender:
demo_gender
Female    59101
Male      46526
Name: count, dtype: int64

Counts for demo_race:
demo_race
White or Caucasian                           51082
Black or African American                    34317
Hispanic or Latino                           19405
Asian                                          660
American Indian or Alaska Native               104
Native Hawaiian or Other Pacific Islander       59
Name: count, dtype: int64

Counts for demo_employstatus:
demo_employstatus
Retired                    42205
Not Employed               22417
Disabled                   21139
Full Time                  13310
Part Time                   4538
Self Employed               1546
Student - Full Time          402
Student - Part Time           66
On Active Military

Age 
 - is ready to be dummied. No further processing is required.

Gender
- needs to be binarized, Male = 1 and Female = 0 

Race
- needs to be binarized, White = 1 and Person_of_Colour = 0

EmployStatus 
- We will change the groupings to be Employed (both full,part time, Self Employed, and military), Student (both full and part time), Not Employed (includes Not Employed, Disabled, and Retired)
- These will then be dummied

Insurance Status
- We will change the groupings to be the following: Federal(includes Medicare and Medicaid), Private(includes Commercial, Other, and Self-Pay)

---
Let's start with Age, though nothing needs to be done. We sill need to dummy these columns. The instance with the highest occurence will be dropped to avoid multicolinearity. In this case, it will be 50-59. This will also be a reference point

In [100]:
(health_data['demo_age'].value_counts()/health_data['demo_age'].count())*100

demo_age
50-59      22.621110
60-69      21.051436
70-79      16.983347
40-49      12.390771
80-89      12.325447
30-39       7.002944
90-99       4.471395
18-29       3.024795
100-109     0.128755
Name: count, dtype: float64

In [101]:
# Create dummy variables
dummy_age = pd.get_dummies(health_data['demo_age'], prefix='demo_age')

# Concatenate the dummy variables with the original DataFrame
health_data = pd.concat([health_data, dummy_age], axis=1)

In [102]:
#identify dummy columns
dummy_age = health_data.filter(like='demo_age_').columns 

# Identify the column with the most instances
most_instances_column = health_data[dummy_age].sum().idxmax()

print(f'The dummy column with the most instances is: {most_instances_column}')

The dummy column with the most instances is: demo_age_50-59


In [103]:
# Drop the original column with the most instances
health_data = health_data.drop(most_instances_column, axis=1)

In [104]:
#Drop the original column 
health_data = health_data.drop(columns='demo_age')

---
With Gender, we will binarize with Male = 1 and Female = 0

In [105]:
(health_data['demo_gender'].value_counts()/health_data['demo_gender'].count())*100

demo_gender
Female    55.95255
Male      44.04745
Name: count, dtype: float64

In [106]:
#Map the columns accordingly
health_data['demo_gender'] = health_data['demo_gender'].map({'Male': 1, 'Female': 0})

#Ensure the mapping worked accordingly
health_data['demo_gender'].value_counts()

demo_gender
0    59101
1    46526
Name: count, dtype: int64

---
Now moving onto Race, we will binarize with White = 1 and Person_of_Colour = 0

In [107]:
(health_data['demo_race'].value_counts()/health_data['demo_race'].count())*100

demo_race
White or Caucasian                           48.360741
Black or African American                    32.488852
Hispanic or Latino                           18.371250
Asian                                         0.624840
American Indian or Alaska Native              0.098460
Native Hawaiian or Other Pacific Islander     0.055857
Name: count, dtype: float64

In [108]:
#Map the columns accordingly
health_data['demo_race'] = health_data['demo_race'].map({'White or Caucasian': 1, 'Black or African American': 0, 
'Hispanic or Latino':0, 'Asian':0, 'American Indian or Alaska Native':0,'Native Hawaiian or Other Pacific Islander':0})

#Making sure the mapping worked 
health_data['demo_race'].value_counts()

demo_race
0    54545
1    51082
Name: count, dtype: int64

---
Now moving onto employ status, we will change the groupings to be Employed (both full,part time, Self Employed, and military), Student (both full and part time), Not Employed (includes Not Employed, Disabled, and Retired). After we will dummy these columns. We will drop the column with the most instances as per protocol to reduce multicolinearity. This means that the default will be the highest group. In this case, it would be individuals who are not employed.

In [109]:
(health_data['demo_employstatus'].value_counts()/health_data['demo_employstatus'].count())*100

demo_employstatus
Retired                    39.956640
Not Employed               21.222793
Disabled                   20.012875
Full Time                  12.600945
Part Time                   4.296250
Self Employed               1.463641
Student - Full Time         0.380585
Student - Part Time         0.062484
On Active Military Duty     0.003787
Name: count, dtype: float64

In [110]:
#Map the columns accordingly
health_data['demo_employstatus'] = health_data['demo_employstatus'].map({'Full Time': 'Employed', 'Part Time': 'Employed','Self Employed': 'Employed','On Active Military Duty': 'Employed',
'Student - Full Time ': 'Student','Student - Part Time': 'Student',
'Retired': 'Not_Employed','Female': 'Not_Employed','Disabled': 'Not_Employed'})

#Making sure the mapping worked 
health_data['demo_employstatus'].value_counts()

demo_employstatus
Not_Employed    63344
Employed        19398
Student            66
Name: count, dtype: int64

We will now need to convert these into dummy columns

In [111]:
# Create dummy variables
dummy_age = pd.get_dummies(health_data['demo_employstatus'], prefix='demo_employstatus')

# Concatenate the dummy variables with the original DataFrame
health_data = pd.concat([health_data, dummy_age], axis=1)

In [112]:
#identify dummy columns
dummy_age = health_data.filter(like='demo_employstatus_').columns 

# Identify the column with the most instances
most_instances_column = health_data[dummy_age].sum().idxmax()

print(f'The dummy column with the most instances is: {most_instances_column}')

The dummy column with the most instances is: demo_employstatus_Not_Employed


In [113]:
# Drop the original column with the most instances
health_data = health_data.drop(most_instances_column, axis=1)

In [114]:
#Drop the original column 
health_data = health_data.drop(columns='demo_employstatus')

---
Finally, We will change the groupings to be the following: Federal(includes Medicare and Medicaid), Private(includes Commercial, Other, and Self-Pay)

After we will dummy these columns. We will drop the column with the most instances as per protocol to reduce multicolinearity. This means that the default will be the highest group. 

In [115]:
(health_data['demo_insurance_status'].value_counts()/health_data['demo_insurance_status'].count())*100

demo_insurance_status
Medicare      38.352883
Commercial    30.030201
Medicaid      27.797817
Other          3.537921
Self pay       0.281178
Name: count, dtype: float64

In [116]:
#Map the columns accordingly
health_data['demo_insurance_status'] = health_data['demo_insurance_status'].map({'Medicare':'Federal','Medicaid':'Federal','Commercial':'Private','Other':'Private','Self pay':'Private',})

#Making sure the mapping worked 
health_data['demo_insurance_status'].value_counts()

demo_insurance_status
Federal    69873
Private    35754
Name: count, dtype: int64

We will now need to convert these into dummy columns

In [117]:
# Create dummy variables
dummy_age = pd.get_dummies(health_data['demo_insurance_status'], prefix='demo_insurance_status')

# Concatenate the dummy variables with the original DataFrame
health_data = pd.concat([health_data, dummy_age], axis=1)

In [118]:
#identify dummy columns
dummy_age = health_data.filter(like='demo_insurance_status_').columns 

# Identify the column with the most instances
most_instances_column = health_data[dummy_age].sum().idxmax()

print(f'The dummy column with the most instances is: {most_instances_column}')

The dummy column with the most instances is: demo_insurance_status_Federal


In [119]:
# Drop the original column with the most instances
health_data = health_data.drop(most_instances_column, axis=1)

In [120]:
#Drop the original column 
health_data = health_data.drop(columns='demo_insurance_status')

---
Before we can start dummying our data, we first need to clean up the categories. We will do this supercategory by supercategory. Now let's go to `triage_num` supercategory. 

In [121]:
triage_num = health_data.filter(like='triage_vital').columns
print(f'The columns that make up the demographic super category are : {triage_num}')

The columns that make up the demographic super category are : Index(['triage_vital_hr', 'triage_vital_sbp', 'triage_vital_dbp',
       'triage_vital_rr', 'triage_vital_temp', 'triage_vital_hr_imputed_flag',
       'triage_vital_sbp_imputed_flag', 'triage_vital_dbp_imputed_flag',
       'triage_vital_rr_imputed_flag', 'triage_vital_temp_imputed_flag'],
      dtype='object')


We will now need to convert these into dummy columns, 

In [122]:
# Create dummy variables
dummy_age = pd.get_dummies(health_data['demo_insurance_status'], prefix='demo_insurance_status')

# Concatenate the dummy variables with the original DataFrame
health_data = pd.concat([health_data, dummy_age], axis=1)

KeyError: 'demo_insurance_status'

In [ ]:
#identify dummy columns
dummy_age = health_data.filter(like='demo_insurance_status_').columns 

# Identify the column with the most instances
most_instances_column = health_data[dummy_age].sum().idxmax()

print(f'The dummy column with the most instances is: {most_instances_column}')

The dummy column with the most instances is: demo_insurance_status_Federal


In [ ]:
# Drop the original column with the most instances
health_data = health_data.drop(most_instances_column, axis=1)

In [ ]:
#Drop the original column 
health_data = health_data.drop(columns='demo_insurance_status')